<h2>Importing Libraries</h2>

In [ ]:
# pip install plotly

In [ ]:
#linear algebra
import pandas as pd
import numpy as np
#visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import iplot
#statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import het_goldfeldquandt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import pearsonr

<h2>Loading the dataset</h2>

In [ ]:
df=pd.read_csv("../input/ford-used-car-listing/ford.csv")
df.head(3)

<h2>Data Pre-processing</h2>

In [ ]:
df.isnull().sum()

<p>No null values in the dataset</p>

<h2>Data Visualization</h2>

In [ ]:
df.dtypes

<h2>Visualization for target feature</h2>

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(9,9))
sns.boxplot(x="price",data=df,color="aqua")
plt.xlabel("Price",fontsize=14)
plt.title("Box plot for Price",fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(9,9))
sns.histplot(x=df["price"], color="purple", label="100% Equities", kde=True, stat="density", linewidth=0)
plt.xlabel("Price",fontsize=14)
plt.ylabel("Density",fontsize=14)
plt.title("Distribution plot for Price",fontsize=14)
plt.show()

<p>Area of density curve is very high from 0 to ~~25000. Looks like probability that a price lying in that region is ~~0.98. We will confirm that.</p>

In [ ]:
print(df[df['price']>25000].shape)
print(df.shape)

<p>approximately 1.36 percent of the data lies above 17965. So, we were kind of correct.Now, we will remove the supposed outliers.</p>

In [ ]:
df=df[df['price']<=25000]
df.head()

In [ ]:
plt.figure(figsize=(9,9))
sns.histplot(x=df["price"], color="purple", label="100% Equities", kde=True, stat="density", linewidth=0)
plt.xlabel("Price",fontsize=14)
plt.ylabel("Density",fontsize=14)
plt.title("Distribution plot for Price",fontsize=14)
plt.show()

<p>Now it looks a little less skewed and close to normal shape (</p>

<h2>Visualization for Categorical features</h2>

In [ ]:
categorical=[feature for feature in df.columns if df[feature].dtype =="object"]
print(categorical)
df_categorical=df[categorical]


<h2>Bar plot for categorical features</h2>

In [ ]:
plt.figure(figsize=(20,20))
sns.countplot(x='model',data=df_categorical)
plt.xticks(rotation=45,fontsize=12)
plt.xlabel("Car Model",fontsize=20)
plt.ylabel("Frequency",fontsize=20)
plt.show()

In [ ]:
df.head()
print(df.shape)
df['model'].value_counts()

In [ ]:
#bar plot for transmission
plt.figure(figsize=(9,9))
sns.countplot(x='transmission',data=df_categorical)
plt.xticks(rotation=45,fontsize=12)
plt.xlabel("Transmission Type",fontsize=14)
plt.ylabel("Frequency",fontsize=14)
plt.show()

<p>Most cars are manual transmission type </p>

In [ ]:
#bar plot for fuel type
#bar plot for transmission
plt.figure(figsize=(9,9))
sns.countplot(x='fuelType',data=df_categorical,palette="tab10")
plt.xticks(rotation=45,fontsize=12)
plt.xlabel("Transmission Type",fontsize=14)
plt.ylabel("Frequency",fontsize=14)
plt.show()

<p>Insufficient samples for Hybrid, Electric and Other. Maybe creating more such samples might help.</p>

In [ ]:
# redundant_fuelTypes=["Hybrid","Electric","Other"]
# redundant_fuelTypes=pd.Series(redundant_fuelTypes)
# df=df[~df['fuelType'].str.contains("|".join(redundant_fuelTypes))]

In [ ]:
df['fuelType'].value_counts()

<h2>Bar plot for year of manufacture</h2>

In [ ]:
#bar plot for fuel type
#bar plot for transmission
plt.figure(figsize=(9,9))
sns.countplot(x='year',data=df,palette="Spectral")
plt.xticks(rotation=45,fontsize=12)
plt.xlabel("Year",fontsize=14)
plt.ylabel("Frequency",fontsize=14)
plt.show()

<p>Most data is 2007 onwards</h2>

In [ ]:
# df=df[df['year']>=2007]
# print(df.shape)
# df.head()

<h2>Pie Chart for  Year distribution</h2>

In [ ]:
fig = {
  "data": [
    {
     
      "labels": df['year'],
      "domain": {"x": [0, .5]},
      "name": "Number Of Cars",
      "hoverinfo":"label+percent+name",
      "hole": .3,
      "type": "pie"
    },],
  "layout": {
        "title":"Manufacturing years of the cars",
        "annotations": [
            { "font": { "size": 20},
              "showarrow": False,
              "text": "Number of cars",
                "x": 0.20,
                "y": 1.2
            },
        ]
    }
}
iplot(fig)

<h2>Pie Chart for transmission type</h2>

In [ ]:
fig = {
  "data": [
    {
     
      "labels": df['transmission'],
      "domain": {"x": [0, .5]},
      "name": "Number Of Cars",
      "hoverinfo":"label+percent+name",
      "hole": .3,
      "type": "pie"
    },],
  "layout": {
        "title":"Transmission Type of the Cars",
        "annotations": [
            { "font": { "size": 20},
              "showarrow": False,
              "text": "Number of cars",
                "x": 0.20,
                "y": 1.1
            },
        ]
    }
}
iplot(fig)

<h2>Pie Chart for Car Model</h2>

In [ ]:
fig = {
  "data": [
    {
     
      "labels": df['model'],
      "domain": {"x": [0, .5]},
      "name": "Number Of Cars",
      "hoverinfo":"label+percent+name",
      "hole": .3,
      "type": "pie"
    },],
  "layout": {
        "title":"Car Models",
        "annotations": [
            { "font": { "size": 20},
              "showarrow": False,
              "text": "Number of cars",
                "x": 0.20,
                "y": 1.2
            },
        ]
    }
}
iplot(fig)

<h2>One-hot encoding</h2>

In [ ]:
# to avoid dummy variable trap, we will be removing the first categorical variable
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]],drop_first=True)
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

data_test = encode_and_bind(df,"transmission")
data_test.head()

In [ ]:
#performing one hot encoding for all categorical variables

categorical = [ feature for feature in df.columns if df[feature].dtype =="object" ]

for idx , feature in enumerate(categorical) :
    
    if idx == 0 :
        
        training_set = encode_and_bind(df,feature)
        continue
    
    training_set = encode_and_bind(training_set,feature)
    

training_set.rename(columns={"model_ Ka+":"model_Ka_plus"},inplace=True)
#remove spaces in column names
training_set.columns = training_set.columns.str.replace(" ","_")
training_set.columns = training_set.columns.str.replace("-","_")
training_set.head()




In [ ]:
plt.figure(figsize=(9,9))
sns.stripplot(x="year",y="price",data=df)
plt.xticks(rotation=45,fontsize=12)
plt.xlabel("Year",fontsize=14)
plt.ylabel("Price",fontsize=14)
plt.show()


<p> Looks like the average price conditional on year increases with year. So, we will include this in our regression model</p>

<h2> Performing ordinary least squares regression </h2>

In [ ]:
Y = training_set['price']
X = training_set.drop(labels=["price"],axis=1)

In [ ]:
X.head()

In [ ]:
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()

<h3> Condition number > 1000 , so there is  severe multicollinearty among variables </h2>

<h2> Calculating V.I.F </h2>

In [ ]:
#gather features
f_list = training_set.columns.tolist()
f_list.remove("price")

features = "+".join(f_list)
print(features)

# get y and X dataframes based on this regression:
y_dash, X_dash = dmatrices('price ~{}'.format(features), training_set, return_type='dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_dash.values, i) for i in range(X_dash.shape[1])]
vif["features"] = X_dash.columns

In [ ]:
vif.round(1)

<p>Variables with high V.I.F :- model__Fiesta,model__Focus</p>

<h3> Dropping model__Fiesta and performing OLS </h3>

In [ ]:
Y = training_set['price']
X = training_set.drop(labels=["price","model__Fiesta"],axis=1)

#perform ols
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()

<h3> Dropping model__Focus and performing OLS </h3>

In [ ]:
Y = training_set['price']
X = training_set.drop(labels=["price","model__Focus"],axis=1)

#perform ols
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()

<h3> discarding model__Fiesta for high r squared values </h3>

In [ ]:
training_set_new = training_set.drop(labels=["price","model__Fiesta"],axis=1)

#run OLS
X = training_set_new.copy()
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()



 Now, level of significance is 0.05 or rejection region in right-tailed test has p-value of less than or equal to 5 percent
model__C_MAX, fuelType_Electric,fuelType_Other and model__Transit_Tourneo has p value greater than 0.05.
$$ H_0 : \hat{\beta_i} = 0 $$
$$ H_1 : \hat{\beta_i} \neq 0 $$
 If  p-value <= 0.05 , we reject $ H_0 $. This means that the regression parameter for that independent variable does determine the independent variable.
 Else, we accept $H_0$ that the variable is not significant for regression

<h3> Discarding model__C_MAX ,  fuelType_Electric,fuelType_Other and model__Transit_Tourneo </h2>

In [ ]:
training_set_new = training_set_new.drop(labels=["model__C_MAX","fuelType_Electric","fuelType_Other","model__Transit_Tourneo"],axis=1)

#run OLS
X = training_set_new.copy()
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()


<h2> Dropping tax </h2>

In [ ]:
training_set_new = training_set_new.drop(labels=["tax"],axis=1)

#run OLS
X = training_set_new.copy()
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()

Looks like now all the variables are statistically significant

<h2> Calculating VIF for the new model </h2>

In [ ]:
#gather features
f_list = training_set_new.columns.tolist()

features = "+".join(f_list)
print(features)

# get y and X dataframes based on this regression:
y_dash, X_dash = dmatrices('price ~{}'.format(features), training_set, return_type='dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_dash.values, i) for i in range(X_dash.shape[1])]
vif["features"] = X_dash.columns

In [ ]:
vif.round(1)

<p> So, now we have low VIF's. This means according to V.I.F test, there is no multicollinearity in the model</p>

<h1 style="color:red;"> Checking assumptions for Classical Linear Regression</h1>

<h2> <u>Assumption 1</u> :-Linearity in Parameters</h2>

<p style="color:green;"> OLS is a linear least square estimator so satisfied</p> 


<h2> <u>Assumption 2</u> :- X values are non-stochastic.</h2>
<p style="color:green;"> Ols assumes this </p>


<h2> <u>Assumption 3</u> :- $E(u_i|X_i) = 0$</h2>
Softer version of this assumption is that $E(u_i)= 0.$ This means that the sum of all residuals in pop is zero. We can solve this using t-test on sample mean.

Another assumption of this is that $cov(u_i,X_i) = 0$. This we can simply find by correlation


In [ ]:
y_hat = model.predict(X)
u_hat = Y - y_hat

#checking normality of error terms
plt.figure(figsize=(9,9))
sns.histplot(x=u_hat, color="purple", label="100% Equities", kde=True, stat="density", linewidth=0)
plt.xlabel("u_hat",fontsize=14)
plt.ylabel("P(u_hat)",fontsize=14)
plt.title("Distribution plot for error terms",fontsize=14)
plt.show()

<p> So, error terms are normally distributed. We can assume the same for population too. So, we can use parametric tests</p>

<h3> Sample mean </h3>

In [ ]:
x_bar = np.mean(u_hat)


<h3> Sample standard deviation </h3>

In [ ]:
s = np.std(u_hat)*len(u_hat)
s /= (len(u_hat) - 1)
print(s)

<h3>Standard error </h3>

In [ ]:
import math

std_err = s / math.sqrt(len(u_hat))

print(std_err)

<h2> Test-statistic </h2>
<br>
$H_0 : \mu = 0$
<br/>
$H_1: \mu \neq 0 $

In [ ]:
t = (x_bar - 0) / std_err
print(t)

In [ ]:
print(format(-8.31251760610854e-10,'.15f'))

In [ ]:
from scipy.stats import ttest_1samp

ttest_1samp(u_hat,0)

<p> This value is symmetrical as student's t-distribution is symmetrical about the mean. </p>

<p style="color:green;">Now p-value is 0.999 or 99 percent, so at alpha =0.05 or 5 % signifiance, we can accept the null hypothesis by two-tailed t-test.</p>

Therefore, $u=0$.

Or, we can say that population mean of errors is 0. or $E(u_i) = 0$.

<h3> No correlation between independent varibles and error terms</h3>

In [ ]:


ind_vars= list(training_set_new.columns)

corr_df = pd.DataFrame()

for variable in ind_vars:
    
    corr = pearsonr(u_hat,training_set_new[variable])[0]
    corr_df[variable]=[corr]

corr_df

<p style="color:green;"> As, the error term is not at all correlated with any of the features, we have second condition satisfied too.
So, we can say that assumption 3 is satisfied.</p>

<h2> <u>Assumption 4</u> :- $ var(u_i|X_i) = \sigma^2 $. Or homoscedasticity is present</h2>

This assumption means that conditional variance is constant throughout the model.


<h3>Residual plot</h3>

In [ ]:
u_hat_square = np.square(u_hat)
d = pd.DataFrame()
d['residuals'] = u_hat_square
d['price'] = y_hat

plt.figure(figsize=(9,9))
a_plot = sns.scatterplot(x="price",y="residuals",data=d)
plt.ylabel("u_hat_square",fontsize=14)
plt.title("Residual plot",fontsize=14)
plt.xlabel("Y_hat",fontsize=14)
plt.show()



<p> Graphically, for positive values the error remains pretty constant. No heteroscedasticity graphically. </p>


<h3>1. Goldfeld Quandt test</h3>

In [ ]:

#perform goldfeld quandt test
name = ["F statistic", "p-value"]
test = het_goldfeldquandt(model.resid, model.model.exog)
lzip(name, test)

<h2> F-test </h2>
<br>
$H_0 :$  Homoscedasticity is present
<br>
$H_1:$ No Homoscedasticity

Now, Ftab=1

Ftab < Fcalc, So test statistic lies in the critical region.
Therefore, we reject null-hypothesis.
Therefore, heteroscedasticity is present.

<h3>2. Breusch Pagan Godfrey test</h3>

In [ ]:
name = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
test = het_breuschpagan(model.resid, model.model.exog)
lzip(name, test)

<p style="color:red;"> Againm by f-test, heteroscefasticity is very likely </p>

<h2> <u>Assumption 5</u>: No multicollinearity</h2>
    <p style="color:green;"> Already proved that multicollinearity is not present.</p>

<h2> <u>Assumption 6</u>: More observations than parameters to be estimated</h2>
<p style="color:green;"> Already satisfied </p>


<h1> Gauss Markov Validation </h1>

- <i style="color:green; font-size:20px">&#x2611;</i> Model is linear
- <i style="color:green; font-size:20px">&#x2611;</i> Model is unbiased i.e $E(\hat{\beta_2}) = \beta_2$
- <i style="color:red; font-size:20px">&#9746;</i> In this model all estimators do no assume minimum variance. An unbiased estimator with minimum variance is efficient estimator. But in our case, due to presence of heteroscedasticity.


### So, our model is not BLUE estimator in this case.

If we persist on using usual testing procedures despite heteroscedasticity, whatever conclusions or inferences that we draw might be misleading.


<h1> Remedies ?</h1>

1> As mentioned earlier in presence of heteroscedastic residuals the ordinary least square estimates no longer has minimum variance. However, we can still continue with our regression model if we can address the issue of incorrect standard errors in order to make our interval estimates and hypothesis tests valid. We can do this by using robust standard errors. The robust standard errors addresses the issue of having incorrect interval estimates due to erroneous standard error. However, even with robust standard error the estimates will no longer be minimum variance but we can be okay with it if we have a large enough sample. 

2> We can also try other regression like GLS.

<p> Comment if you find anything misleading or incorrect.  I am new to statistics so there might be a lot of errors.</p>